In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read input

In [2]:
import pandas as pd
import numpy as np
import math
df = pd.read_csv('/content/drive/MyDrive/Fall 2021/EE 454/Lab4/google-cluster-data-1.csv',sep=' ')

In [3]:
df

,Time,ParentID,TaskID,JobType,NrmlTaskCores,NrmlTaskMem,Unnamed: 6
0,90000,757745334,1488529826,0,0.000000,0.031130,NaN
1,90000,975992247,1488529821,0,0.000000,0.000000,NaN
2,90000,1468458091,1488529832,1,0.021875,0.002353,NaN
3,90000,1460281235,1488529840,0,0.000000,0.000000,NaN
4,90000,1164728954,1488529835,0,0.003125,0.001638,NaN
...,...,...,...,...,...,...,...
3535024,112500,1487094655,1487103476,0,0.000000,0.000879,NaN
3535025,112500,1461321601,1465612301,0,0.000000,0.000879,NaN
3535026,112500,1487094655,1487097223,0,0.000000,0.000879,NaN
3535027,112500,618817162,1485932004,1,0.000000,0.000879,NaN


In [4]:
# Constants that change between rounds
burst_time_mod_operand_1 = 2
burst_time_mod_operand_2 = 10
cpu_max_1 = 7
cpu_max_2 = 25
mem_max_1 = 11
mem_max_2 = 40
dynamic_pwr_threshold_1 = 0.5
dynamic_pwr_threshold_2 = 0.9


In [5]:
# Constants that are constant between rounds
c=5 
a=100
b=200
price_model = [0.5, 0.5, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8]
time_zero = 90000
times = set(df['Time'])   # set version of df["Time"] so there are no duplicates



## Round-Robin Alg
Assumption 1: we search from first VM.


Assumption 2: search from i+1 instead of j+1.

In [6]:
def energy_consumption_per_task(usage_rate, dynamic_pwr_threshold, a, b, c):
  # sum of static power consumption and dynamic power consumption
  pwr_static = c if dynamic_pwr_threshold > 0 else 0
  # if dynamic_pwr_threshold > 0:
  #   pwr_static = c
  # else:
  #   pwr_static = 0
  if usage_rate < dynamic_pwr_threshold:
    pwr_dynamic = usage_rate * a
  else:   # usage rate is greater than or equal to threshold
    pwr_dynamic = (dynamic_pwr_threshold*a) + (((usage_rate - dynamic_pwr_threshold)**2) * b)
  return pwr_static + pwr_dynamic

In [7]:
def price(time):
  loc = ((time - time_zero) // 3600) % 12    # integer division to round down for the hour
  return price_model[loc]

In [8]:
def rr(round, dynamic_pwr_threshold, burst_time_mod_operand, cpu_max, mem_max):
  rejected = []
  subtasks = 0
  total_energy = 0.0
  total_cost = 0.0

  # First column is CPU units, second is MEM
  # insert column for burst time AKA execution time
  VMs = [[cpu_max, mem_max] for i in range(100)]
  df['ExecutionTime_'+ round] = ((df['TaskID']%burst_time_mod_operand)+1)*300
  subtasks = 0
  for i in range(len(df)):
    # subtasks is essentially the number of quantums
    subtasks += math.ceil(df['ExecutionTime_' + round][i] / 300)

  print("Calculated number of subtasks round " + round + ":", subtasks)


  for t in times:
    pwr_t = 0.0
    # only look at task id if the time is the same
    gen = (x for x in df.index if df['Time'][x] == t) # generator expression to improve speed
    for i in gen:  
      cpu, mem = df['NrmlTaskCores'][i], df['NrmlTaskMem'][i]
      VM_id = i % 100
      foundAvailableVM = -1
      if VMs[VM_id][0] >= cpu and VMs[VM_id][1] >= mem:      # found an available VM
        foundAvailableVM = VM_id
        ## update VM resource availability. 
        VMs[VM_id][0] = VMs[VM_id][0] - cpu   # subtract out used cpus for this task 
        VMs[VM_id][1] = VMs[VM_id][1] - mem   # subtract out used mem units for this task
      else:
        ## search next available VM. 
        j = VM_id
        while True:
          if (VM_id == 0 and j == 99) or (VM_id != 0 and j == VM_id-1):
            break

          if VMs[j][0] >= cpu and VMs[j][1] >= mem:      
            # found an available VM
            foundAvailableVM = j
            # update resources availability 
            VMs[j][0] = VMs[j][0] - cpu   # subtract out used cpus for this task  
            VMs[j][1] = VMs[j][1] - mem   # subtract out used mem units for this task
            break
          
          if j == 99:
            j = 0
          else:
            j += 1
          subtasks += 1

        if foundAvailableVM == -1:
          rejected.append(df['TaskID'][i])
        else:
          # don't want to calculate energy consumption for rejected tasks because we don't have an available VM to use to calculate usage rate
          usage_rate = cpu / VMs[foundAvailableVM][0]
          
          pwr_t += energy_consumption_per_task(usage_rate, dynamic_pwr_threshold, a, b, c)

    total_energy += pwr_t
    cost = price(t) * pwr_t
  
  print("Number of rejected tasks:", len(rejected))
  if len(rejected) > 0:
    np.save('taskReject_' + round + '.npy', rejected)
  rejectionRate = 100 * len(rejected) / 3535029
  print("Rejection rate: ", end="")
  print(f'{rejectionRate:.2f} %')

  print("Counted number of subtasks:", subtasks)

  print("\nTotal energy consumed:", total_energy)
  print("\nTotal cost:", total_cost)
  
  np.save('exe_time_' + round + '.npy', df['ExecutionTime_' + round])    # save execution time
  np.save('VMs_' + round + '.npy', VMs)   # save VMs.npy file, showing remaining resources left after processing google cluster data


In [9]:
print("---- Round 1 ----")
rr('1', dynamic_pwr_threshold_1, burst_time_mod_operand_1, cpu_max_1, mem_max_1)



---- Round 1 ----
Calculated number of subtasks round 1: 5298274
Number of rejected tasks: 0
Rejection rate: 0.00 %
Counted number of subtasks: 343341489

Total energy consumed: 21500448.718800142

Total cost: 0.0


In [10]:
print("---- Round 2 ----")
rr('2', dynamic_pwr_threshold_2, burst_time_mod_operand_2, cpu_max_2, mem_max_2)


---- Round 2 ----
Calculated number of subtasks round 2: 19472018
Number of rejected tasks: 0
Rejection rate: 0.00 %
Counted number of subtasks: 339099006

Total energy consumed: 16881067.272083394

Total cost: 0.0


## Give an explanation of why there is or there is no rejection under the set up of 7 cpu and 11 mem.

The VMs are not all running at the same time, so we don't need as many CPU units or MEM units.